In [12]:
url = 'https://m.trademap.org/#/grid/391620/076/0/export/country'

In [14]:
import requests

data = requests.get('https://m.trademap.org/#/grid/391620/076/0/export/country')



In [17]:
BRAZIL_CODE = '076'

In [40]:
import requests

url = "https://m.trademap.org/api/ByCountry"

payload = "{\"language_code\":\"en\",\"trade_type\":\"I\",\"product_type\":6,\"product_code\":\"391620\",\"sort_expression\":\"value desc\"}"
headers = {
  'Accept': 'application/json, text/plain, */*',
  'Accept-Language': 'en-US,en;q=0.6',
  'Connection': 'keep-alive',
  'Content-Type': 'application/json;charset=UTF-8',
  'Origin': 'https://m.trademap.org',
  'Referer': 'https://m.trademap.org/',
  'Sec-Fetch-Dest': 'empty',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-GPC': '1',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)


[{"country_code":"000","country_label":"World","unit":"No quantity","value":3800626.0,"trade_balance":261171.0,"quantity":0.0,"unit_value":null,"trend_value":7.0,"trend_value_2a":34.0,"trend_quantity":null,"share_world":100.0,"distance":1927.0,"concentration":0.17,"tariff":null},{"country_code":"842","country_label":"United States of America","unit":"Tons","value":450052.0,"trade_balance":-341124.0,"quantity":135085.0,"unit_value":3332.0,"trend_value":13.0,"trend_value_2a":38.0,"trend_quantity":9.0,"share_world":11.8,"distance":2654.0,"concentration":0.66,"tariff":2.2},{"country_code":"616","country_label":"Poland","unit":"Tons","value":267894.0,"trade_balance":135050.0,"quantity":88117.0,"unit_value":3040.0,"trend_value":13.0,"trend_value_2a":29.0,"trend_quantity":7.0,"share_world":7.0,"distance":681.0,"concentration":0.81,"tariff":1.7},{"country_code":"276","country_label":"Germany","unit":"Tons","value":258737.0,"trade_balance":1114983.0,"quantity":82363.0,"unit_value":3141.0,"trend

In [20]:
import pandas as pd 
top_importers = pd.DataFrame.from_dict(response.json())

In [47]:
url = "https://m.trademap.org/api/BySpecificCountry"
payload = "{\"language_code\":\"en\",\"trade_type\":\"E\",\"product_type\":6,\"product_code\":\"391620\",\"country_code\":\"076\",\"country_reporting\":\"D\",\"sort_expression\":\"value desc\"}"
headers = {
  'Accept': 'application/json, text/plain, */*',
  'Accept-Language': 'en-US,en;q=0.6',
  'Connection': 'keep-alive',
  'Content-Type': 'application/json;charset=UTF-8',
  'Origin': 'https://m.trademap.org',
  'Referer': 'https://m.trademap.org/',
  'Sec-Fetch-Dest': 'empty',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-GPC': '1',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
}

response = requests.request("POST", url, headers=headers, data=payload)

In [49]:
brazil_exports = pd.DataFrame.from_dict(response.json())

In [34]:
brazil_exports

,country_code,country_label,unit,value,trade_balance,quantity,unit_value,trend_value,trend_value_2a,trend_quantity,share_world,distance,concentration,tariff
0,000,World,No quantity,4061797.0,261171.0,0.0,NaN,8.0,35.0,4.0,100.0,1827.0,0.04,NaN
1,276,Germany,Tons,1373720.0,1114983.0,420128.0,3270.0,6.0,35.0,2.0,33.8,1177.0,0.07,1.7
2,792,Türkiye,Tons,421601.0,402005.0,207536.0,2031.0,12.0,32.0,8.0,10.4,2578.0,0.10,3.2
3,616,Poland,Tons,402944.0,135050.0,150128.0,2684.0,11.0,45.0,6.0,9.9,892.0,0.12,1.7
4,124,Canada,Tons,368107.0,273631.0,108004.0,3408.0,12.0,39.0,9.0,9.1,1187.0,0.98,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,417,Kyrgyzstan,Tons,1.0,-14510.0,0.0,NaN,-77.0,NaN,NaN,0.0,525.0,1.00,6.3
95,384,Côte d'Ivoire,Tons,1.0,-754.0,0.0,NaN,NaN,NaN,NaN,0.0,8901.0,1.00,4.7
96,226,Equatorial Guinea,Tons,1.0,-87.0,0.0,NaN,NaN,NaN,NaN,0.0,1437.0,1.00,9.8
97,178,Congo,Tons,1.0,-3.0,2.0,500.0,NaN,NaN,NaN,0.0,1857.0,1.00,4.9


In [50]:
final_df = top_importers[['country_code', 'country_label', 'value']]

In [52]:
brazil_exports

,reporter_code,partner_code,partner_label,value,trade_balance,share,quantity,quantity_unit_label,unit_value,trend_value,trend_quantity,trend_value_2a,tariff,share_partners,distance,concentration
0,000,000,World,24067.0,5202.0,100.0,8640.0,Tons,2786.0,0.0,0.0,62.0,NaN,100.0,2563.0,0.16
1,076,858,Uruguay,5558.0,5558.0,23.1,2357.0,Tons,2358.0,28.0,26.0,24.0,0.0,0.2,2454.0,0.77
2,076,032,Argentina,5171.0,2281.0,21.5,1709.0,Tons,3026.0,5.0,9.0,47.0,0.0,0.3,6531.0,0.23
3,076,600,Paraguay,4185.0,4177.0,17.4,2035.0,Tons,2057.0,-7.0,-11.0,56.0,0.0,0.1,2806.0,0.71
4,076,068,"Bolivia, Plurinational State of",2174.0,2174.0,9.0,934.0,Tons,2328.0,-10.0,-8.0,194.0,0.0,0.1,3989.0,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,,304,Greenland,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,0.0,3388.0,1.00
194,,308,Grenada,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,2.5,0.0,4128.0,1.00
195,,072,Botswana,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,9.0,0.0,522.0,1.00
196,,776,Tonga,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,15.0,0.0,2264.0,1.00


In [53]:
brazil_exports = brazil_exports[['partner_code','value']]

In [31]:
brazil_exports

,country_code,value
0,000,4061797.0
1,276,1373720.0
2,792,421601.0
3,616,402944.0
4,124,368107.0
...,...,...
94,417,1.0
95,384,1.0
96,226,1.0
97,178,1.0


In [56]:
final_df = pd.merge(final_df, brazil_exports, left_on='country_code', right_on='partner_code', how='left')

In [57]:
final_df

,country_code,country_label,value_x,partner_code,value_y
0,000,World,3800626.0,000,24067.0
1,842,United States of America,450052.0,842,5.0
2,616,Poland,267894.0,616,NaN
3,276,Germany,258737.0,276,44.0
4,642,Romania,201709.0,642,NaN
...,...,...,...,...,...
193,534,Sint Maarten (Dutch part),1.0,534,NaN
194,072,Botswana,1.0,072,NaN
195,849,United States Minor Outlying Islands,1.0,849,NaN
196,882,Samoa,1.0,882,NaN


In [63]:
final_df = final_df[1:11]

In [94]:
import requests

def get_single_importers_data(country_code):
    url = "https://m.trademap.org/api/BySpecificCountry"

    payload = f"{{\"language_code\":\"en\",\"trade_type\":\"I\",\"product_type\":6,\"product_code\":\"391620\",\"country_code\":\"{country_code}\",\"country_reporting\":\"D\",\"sort_expression\":\"value desc\"}}"
    headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.6',
    'Connection': 'keep-alive',
    'Content-Type': 'application/json;charset=UTF-8',
    'Origin': 'https://m.trademap.org',
    'Referer': 'https://m.trademap.org/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-GPC': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
    }

    return requests.request("POST", url, headers=headers, data=payload)

In [105]:
response = get_single_importers_data(368)

In [110]:
response.json()

[]

In [106]:
single_importer = pd.DataFrame.from_dict(response.json())[1:]
single_importer['partner_label'][1]

KeyError: 'partner_label'

In [104]:
final_df

,country_code,country_label,value_x,partner_code,value_y,position_string
1,842,United States of America,450052.0,842,5.0,"Canada/80,9%, China/8,3%, Mexico/2,5%"
2,616,Poland,267894.0,616,NaN,"Germany/90,1%, Belgium/2,7%, Türkiye/1,2%"
3,276,Germany,258737.0,276,44.0,"Poland/49,9%, Netherlands/11,6%, Austria/7,4%"
4,642,Romania,201709.0,642,NaN,"Germany/47,0%, Poland/22,7%, Türkiye/13,1%"
5,251,France,198149.0,251,NaN,"Germany/51,5%, Belgium/19,1%, Poland/11,1%"
6,381,Italy,167506.0,381,NaN,"Germany/78,1%, France/6,4%, Poland/5,7%"
7,528,Netherlands,146011.0,528,NaN,"Germany/67,3%, Belgium/23,2%, Area Nes/2,0%"
8,203,Czech Republic,114131.0,203,NaN,"Germany/77,1%, Poland/11,7%, Italy/2,2%"
9,368,Iraq,112268.0,368,NaN,"Canada/80,9%, China/8,3%, Mexico/2,5%"
10,124,Canada,94476.0,124,NaN,"Canada/80,9%, China/8,3%, Mexico/2,5%"


In [111]:
for country_code in final_df['country_code']:
    try:
        response = get_single_importers_data(country_code)
        single_importer = pd.DataFrame.from_dict(response.json())[1:]

        single_importer['position'] = single_importer.reset_index().index

        first_position_name = single_importer['partner_label'][1]
        second_position_name = single_importer['partner_label'][2]
        third_position_name = single_importer['partner_label'][3]

        first_position_pct = str(single_importer['share'][1]).replace('.',',')
        second_position_pct = str(single_importer['share'][2]).replace('.',',')
        third_position_pct = str(single_importer['share'][3]).replace('.',',')

        position_string = f"{first_position_name}/{first_position_pct}%, {second_position_name}/{second_position_pct}%, {third_position_name}/{third_position_pct}%"
        
        
        final_df.loc[final_df['country_code'] == country_code, 'position_string'] = position_string
    except:
        pass
    print(country_code)

c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


842
616
276
642
251
381
528
203
368
124


In [112]:
final_df.loc[final_df['country_code'] == country_code, 'position_string'] = position_string

,country_code,country_label,value_x,partner_code,value_y,position_string
1,842,United States of America,450052.0,842,5.0,"Canada/80,9%, China/8,3%, Mexico/2,5%"
2,616,Poland,267894.0,616,NaN,"Germany/90,1%, Belgium/2,7%, Türkiye/1,2%"
3,276,Germany,258737.0,276,44.0,"Poland/49,9%, Netherlands/11,6%, Austria/7,4%"
4,642,Romania,201709.0,642,NaN,"Germany/47,0%, Poland/22,7%, Türkiye/13,1%"
5,251,France,198149.0,251,NaN,"Germany/51,5%, Belgium/19,1%, Poland/11,1%"
6,381,Italy,167506.0,381,NaN,"Germany/78,1%, France/6,4%, Poland/5,7%"
7,528,Netherlands,146011.0,528,NaN,"Germany/67,3%, Belgium/23,2%, Area Nes/2,0%"
8,203,Czech Republic,114131.0,203,NaN,"Germany/77,1%, Poland/11,7%, Italy/2,2%"
9,368,Iraq,112268.0,368,NaN,"Canada/80,9%, China/8,3%, Mexico/2,5%"
10,124,Canada,94476.0,124,NaN,"United States of America/70,5%, China/16,9%, G..."


In [73]:
single_importer['position'] = single_importer.reset_index().index

In [85]:
final_df

,country_code,country_label,value_x,partner_code,value_y
1,842,United States of America,450052.0,842,5.0
2,616,Poland,267894.0,616,NaN
3,276,Germany,258737.0,276,44.0
4,642,Romania,201709.0,642,NaN
5,251,France,198149.0,251,NaN
6,381,Italy,167506.0,381,NaN
7,528,Netherlands,146011.0,528,NaN
8,203,Czech Republic,114131.0,203,NaN
9,368,Iraq,112268.0,368,NaN
10,124,Canada,94476.0,124,NaN


In [87]:

final_df.loc[final_df['country_code'] == '842', 'position_string'] = position_string

c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [88]:
final_df

,country_code,country_label,value_x,partner_code,value_y,position_string
1,842,United States of America,450052.0,842,5.0,"Canada/80,9%, China/8,3%, Mexico/2,5%"
2,616,Poland,267894.0,616,NaN,NaN
3,276,Germany,258737.0,276,44.0,NaN
4,642,Romania,201709.0,642,NaN,NaN
5,251,France,198149.0,251,NaN,NaN
6,381,Italy,167506.0,381,NaN,NaN
7,528,Netherlands,146011.0,528,NaN,NaN
8,203,Czech Republic,114131.0,203,NaN,NaN
9,368,Iraq,112268.0,368,NaN,NaN
10,124,Canada,94476.0,124,NaN,NaN
